# Test notebook for CES androids

In [1]:
using Pkg
Pkg.activate("../")

  Activating project at `~/workspace/ExchangeMarket.jl/scripts`


In [2]:
using Revise
using Random, SparseArrays, LinearAlgebra
using JuMP, MosekTools
using Plots, LaTeXStrings, Printf
import MathOptInterface as MOI

using ExchangeMarket

include("../tools.jl")
include("../plots.jl")
switch_to_pdf(; bool_use_html=true)

include("./util.jl")

┌ Warning: `PlotlyKaleido` 2.3.0 is not compatible with this version of `Plots`. The declared compatibility is 1.
└ @ Plots /Users/brent/.julia/packages/Plots/FFuQi/src/backends.jl:55


Cardinal Optimizer v7.2.4. Build date Dec  6 2024
Copyright Cardinal Operations 2024. All Rights Reserved


_linear_prog_ces_gamma_single

## CES economy in linear conic form


In [3]:
m = 3
n = 10
ρ = 0.8
b = rand(n, m)
b ./= sum(b; dims=2)
e = ones(n)

# -----------------------------------------------------------------------
# setup market
# -----------------------------------------------------------------------
f0 = FisherMarket(m, n; ρ=ρ * ones(m), scale=30.0, sparsity=0.99)
linconstr = LinearConstr(1, n, ones(1, n), [1.0])

# -----------------------------------------------------------------------
# compute ground truth
# -----------------------------------------------------------------------
f1 = copy(f0)
p₀ = ones(n) ./ (n)
x₀ = ones(n, m) ./ m
f1.x .= x₀
alg = HessianBar(
    n, m, p₀;
    linconstr=linconstr,
)
alg.linsys = :direct

FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.0533 seconds
FisherMarket initialized in 0.0825 seconds
FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.0002 seconds
FisherMarket initialized in 0.0003 seconds


:direct

In [4]:
play!(alg, f1)
grad!(alg, f1)
eval!(alg, f1)
hess!(alg, f1)

┌ Info: exact dense Hessian built (heterogeneous σ)
└ @ ExchangeMarket /Users/brent/workspace/ExchangeMarket.jl/src/algorithms/diff.jl:223


### Conic Form

In [5]:
idx = 1
cr = f1.c[:, idx] .^ (1 / ρ)
xc = _conic_ces_primal(;
    n=f1.n,
    w=f1.w[idx],
    cr=cr,
    p=alg.p,
    ρ=f1.ρ[1],
    verbose=true
)

Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2               
  Affine conic cons.     : 11 (33 rows)
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 22              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes        

10-element Vector{Float64}:
 0.2703191033148266
 0.21966380702254762
 1.091401761881452e-7
 0.004380051354384578
 0.4953728546867976
 0.013744912814820416
 0.26035596678473283
 0.040795684869418244
 7.352064074075352e-7
 4.5504539418254865e-5

In [6]:
# should be the same
[xc f1.x[:, idx]]

10×2 Matrix{Float64}:
 0.270319    0.270339
 0.219664    0.219626
 1.0914e-7   4.28975e-14
 0.00438005  0.00437742
 0.495373    0.495373
 0.0137449   0.013743
 0.260356    0.260379
 0.0407957   0.0407956
 7.35206e-7  3.23634e-7
 4.55045e-5  4.47504e-5

# Exact recovery from the bidding vector

In [7]:
include("./util.jl")

_linear_prog_ces_gamma_single

## Exact formula of CES p to c, given 1

In [8]:
γ = alg.p .* f1.x[:, 1] ./ f1.w[1]
pmat = zeros(n, 1)
gmat = zeros(n, 1)
pmat[:, 1] .= alg.p
gmat[:, 1] .= γ

# -----------------------------------------------------------------------
# try if I have only one gamma
y, δ, A, md = _linear_prog_ces_gamma_single(pmat=pmat, gmat=gmat, verbose=true)
objective_value(md)

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : LO (linear optimization problem)
  Constraints            : 30              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 23              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 11
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep.  - dual attempts          : 0            

0.0

## Exact formula of CES p to c, given many, but provided $\gamma_1,...$

- here I gave K p-gamma pair
- then use one ces agent to recover
- because it is generated from a ces player,
- it should be recoverable

In [9]:
K = 12
gmat = zeros(n, K)
pmat = zeros(n, K)
zmat = zeros(n, K)
# use the first play
idx = 1
for k = 1:K
    alg.p .= rand(n)
    play!(alg, f1)
    grad!(alg, f1)
    eval!(alg, f1)
    pmat[:, k] .= alg.p
    gmat[:, k] .= alg.p .* f1.x[:, idx] ./ sum(alg.p .* f1.x[:, idx])
    zmat[:, k] .= sum(f1.x, dims=2)
end

In [10]:
y, δ, A, md = _linear_prog_ces_gamma_single(pmat=pmat, gmat=gmat, δ₁=nothing, verbose=true)
@info "" δ f1.σ
γv(p, a) = begin
    γ = exp.(y) .* (p .^ (-δ))
    γ ./= exp(a)
end
γfit = zeros(n, K)
for k = 1:K
    γfit[:, k] .= γv(pmat[:, k], A[k])
end
@info "optimal value" objective_value(md)
@info "tightness" (gmat - γfit) .|> abs |> maximum

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : LO (linear optimization problem)
  Constraints            : 360             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 144             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Eliminator started.
Freed constraints in eliminator : 141
Eliminator terminated.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes    

┌ Info: 
│   δ = 4.000000000501259
│   f1.σ = [4.000000000000001, 4.000000000000001, 4.000000000000001]
└ @ Main /Users/brent/workspace/ExchangeMarket.jl/scripts/revealed/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:2
┌ Info: optimal value
│   objective_value(md) = 7.001924610436284e-10
└ @ Main /Users/brent/workspace/ExchangeMarket.jl/scripts/revealed/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:11
┌ Info: tightness
│   (gmat - γfit .|> abs) |> maximum = 1.0720740961644992e-9
└ @ Main /Users/brent/workspace/ExchangeMarket.jl/scripts/revealed/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:12


In [11]:
gmat ./ γfit

10×12 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

# Test if primal dual master problem is accurate

- generate $\Xi$ from a FisherMarket with CES agents
- recover so from $\gamma$, using the same market (assuming we dont know the budgets)
- the recoverd budgets should agree with the original market because we use the same information

In [12]:
m = 3
n = 10
ρ = 0.8
b = rand(n, m)
b ./= sum(b; dims=2)
e = ones(n)

# -----------------------------------------------------------------------
# setup market
# -----------------------------------------------------------------------
f0 = FisherMarket(m, n; ρ=ρ * ones(m), scale=30.0, sparsity=0.99)
linconstr = LinearConstr(1, n, ones(1, n), [1.0])

# -----------------------------------------------------------------------
# compute ground truth
# -----------------------------------------------------------------------
f1 = copy(f0)
p₀ = ones(n) ./ (n)
x₀ = ones(n, m) ./ m
f1.x .= x₀
alg = HessianBar(
    n, m, p₀;
    linconstr=linconstr,
)
alg.linsys = :direct

FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.0001 seconds
FisherMarket initialized in 0.0001 seconds
FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.0000 seconds
FisherMarket initialized in 0.0001 seconds


:direct

In [13]:
include("./master.jl")
include("./pricing.jl")

recover_ces_params

In [14]:
# Generate revealed preferences from the market
K = 12
Ξ = produce_revealed_preferences(alg, f1, K; seed=42)

# Compute bidding matrix from market parameters
γ = compute_gamma_from_market(f1, Ξ)

w, s, model_primal = solve_master_problem(Ξ, γ; verbose=true)
u, μ, model_dual = solve_dual_problem(Ξ, γ; verbose=true)

# Validate strong duality
println("=== Strong Duality Validation ===")
println("Primal objective (Q):     ", objective_value(model_primal))
println("Dual objective (Q_*):     ", objective_value(model_dual))
println("Gap:                      ", abs(objective_value(model_primal) - objective_value(model_dual)))
println()
println("Primal solution w - ground truth:   ", abs.(w - f1.w) |> maximum)
println("Dual solution μ:          ", μ)
println("Dual solution u:          ", u[1:5])

Set parameter Username
Academic license - for non-commercial use only - expires 2026-06-14
Set parameter OutputFlag to value 1
Set parameter OutputFlag to value 1
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 25.3.0 25D5087f)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 241 rows, 135 columns and 713 nonzeros
Model fingerprint: 0x02cb822d
Coefficient statistics:
  Matrix range     [2e-13, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-07, 1e+00]
Presolve removed 121 rows and 120 columns
Presolve time: 0.00s
Presolved: 120 rows, 15 columns, 470 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.570755e+00   0.000000e+00      0s
       2    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 i

# Test Column-Generation Procedure

- use a underlying market f1, to generate $\Xi$,
- then, create a surrogate market fa, initialized with one agent
- use CG to iteratively add agents

In [15]:
include("./master.jl")
include("./pricing.jl")

recover_ces_params

In [16]:
m = 3
n = 10
K = 12 # how many revealed preferences pairs we have
ρ = 0.8
b = rand(n, m)
b ./= sum(b; dims=2)
e = ones(n)

# -----------------------------------------------------------------------
# setup market
# -----------------------------------------------------------------------
f0 = FisherMarket(m, n; ρ=ρ * ones(m), scale=30.0, sparsity=0.99)
linconstr = LinearConstr(1, n, ones(1, n), [1.0])

# -----------------------------------------------------------------------
# compute ground truth
# -----------------------------------------------------------------------
# Generate revealed preferences from the underlying market
f1 = copy(f0)
p₀ = ones(n) ./ (n)
x₀ = ones(n, m) ./ m
f1.x .= x₀
alg = HessianBar(
    n, m, p₀;
    linconstr=linconstr,
)
alg.linsys = :direct

Ξ = produce_revealed_preferences(alg, f1, K; seed=42);

FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.0001 seconds
FisherMarket initialized in 0.0001 seconds
FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.0000 seconds
FisherMarket initialized in 0.0001 seconds


In [17]:
# Column generation: iteratively add new CES androids to fit revealed preferences
# -----------------------------------------------------------------------
# Parameters
# -----------------------------------------------------------------------
max_iters = 50
tol = 1e-6  # tolerance for reduced cost

# -----------------------------------------------------------------------
# Initialize surrogate market with one random CES agent
# -----------------------------------------------------------------------
fa = FisherMarket(1, n; ρ=rand(1), scale=30.0, sparsity=0.99)
γ_ref = Ref(compute_gamma_from_market(fa, Ξ))

# Track history
history = Dict(
    :primal_obj => Float64[],
    :dual_obj => Float64[],
    :reduced_cost => Float64[],
    :num_agents => Int[]
)

println("=== Column Generation for CES Android Fitting ===\n")

for iter in 1:max_iters
    println("--- Iteration $iter ($(fa.m) agents) ---")

    # Solve master problem (primal and dual)
    w, s, model_primal = solve_master_problem(Ξ, γ_ref[]; verbose=false)
    u, μ, model_dual = solve_dual_problem(Ξ, γ_ref[]; verbose=false)

    primal_obj = objective_value(model_primal)
    dual_obj = objective_value(model_dual)

    push!(history[:primal_obj], primal_obj)
    push!(history[:dual_obj], dual_obj)
    push!(history[:num_agents], fa.m)

    println("  Primal obj: $(round(primal_obj, digits=6))")
    println("  Dual obj:   $(round(dual_obj, digits=6))")
    println("  Weights w:  $(round.(w, digits=4))")

    # Solve pricing subproblem to find best new android
    y_opt, σ_opt, γ_new, pricing_obj = solve_pricing(Ξ, u)

    # Compute reduced cost
    rc = reduced_cost(γ_new, u, μ)
    push!(history[:reduced_cost], rc)

    println("  Pricing σ:  $(round(σ_opt, digits=4))")
    println("  Reduced cost: $(round(rc, digits=6))")

    # Check termination: if reduced cost <= 0, no improving android exists
    if rc <= tol
        println("\n✓ Converged! No improving android found (reduced cost ≤ $tol)")
        break
    end

    # Update existing agents' budgets with optimal weights from master
    fa.w .= w

    # Add new android to γ matrix (in-place via Ref)
    add_to_gamma!(γ_ref, γ_new)

    # Recover CES parameters and add to surrogate market
    c_new, ρ_new = recover_ces_params(y_opt, σ_opt)
    w_new = 0.0  # placeholder, will be updated in next iteration
    add_to_market!(fa, c_new, ρ_new, w_new)

    println("  → Added android $(fa.m) with ρ=$(round(ρ_new, digits=4))")
    println()

    if iter == max_iters
        println("\n⚠ Maximum iterations reached")
    end
end

# Final solve to get optimal weights for all agents
w_final, _, _ = solve_master_problem(Ξ, γ_ref[]; verbose=false)
fa.w .= w_final

println("\n=== Final Results ===")
println("Number of fitted agents: $(fa.m)")
println("Number of nonzero agents: $(sum(w_final .> 1e-6))")
println("Ground truth agents:     $(f1.m)")
println("Final primal objective:  $(round(history[:primal_obj][end], digits=6))")
println("Final weights: $(round.(w_final, digits=4))")

FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.0001 seconds
FisherMarket initialized in 0.0002 seconds
=== Column Generation for CES Android Fitting ===

--- Iteration 1 (1 agents) ---
Set parameter Username
Academic license - for non-commercial use only - expires 2026-06-14
Set parameter Username
Academic license - for non-commercial use only - expires 2026-06-14
  Primal obj: 6.342179
  Dual obj:   6.342179
  Weights w:  [1.0]
  Pricing σ:  49.505
  Reduced cost: 11.360928
  → Added android 2 with ρ=0.9802

--- Iteration 2 (2 agents) ---
Set parameter Username
Academic license - for non-commercial use only - expires 2026-06-14
Set parameter Username
Academic license - for non-commercial use only - expires 2026-06-14
  Primal obj: 1.63633
  Dual obj:   1.63633
  Weights w:  [0.2594, 0.7406]
  Pricing σ:  100.0
  Reduced cost: 6.999998
  → Added android 3 with ρ=0.9901

--- Iteration 3 (3 agents) ---
Set parameter Username
Academic license - for non-co

In [18]:
p1, g1 = Ξ[1]

([0.06712779654090693, 0.08673837419857172, 0.0783854508486898, 0.06618281351094174, 0.11393250590798745, 0.10810670523914472, 0.09587600391262513, 0.15044865550529463, 0.13975130600358215, 0.09345038833225555], [1.1059403784725503, 0.33747753123141383, 0.0011596583904514347, 11.848858568028675, 0.14267575208764446, 0.03467828552603025, 0.5868002779844652, 0.17773409082291228, 0.05507059079734159, 0.016118078912747698])

In [21]:
alg.p .= p1
play!(alg, fa)
ga = sum(fa.x, dims=2)
[ga g1]

10×2 Matrix{Float64}:
 Inf            1.10594
  0.600927      0.337478
  1.37505e-31   0.00115966
 11.7025       11.8489
  0.0576576     0.142676
  2.3189e-8     0.0346783
  0.589157      0.5868
  0.113351      0.177734
  0.0493689     0.0550706
  1.40425e-20   0.0161181

10×41 Matrix{Float64}:
 0.0  5.06607e-31  2.03384e-25  0.237672     …  0.313646      3.22499e-22
 0.0  6.15846e-40  1.49512e-33  3.24739e-24     3.13418e-80   1.67706e-48
 0.0  1.88801e-34  1.37317e-31  1.56616e-46     2.59195e-103  2.00352e-81
 0.0  2.73773      0.55364      3.25695e-28     7.17716e-17   0.103359
 0.0  1.24475e-42  2.2082e-37   5.3035e-23      1.65644e-97   1.52103e-52
 0.0  1.57519e-41  2.33754e-44  6.6455e-55   …  1.51257e-82   1.78463e-45
 0.0  1.12693e-38  5.60018e-7   3.27102e-43     3.84393e-56   4.82402e-32
 0.0  5.36109e-30  8.39375e-22  2.27552e-75     5.43581e-55   3.85882e-28
 0.0  4.84806e-48  1.31974e-31  1.14601e-66     7.67558e-117  3.9346e-48
 0.0  1.18691e-38  2.13371e-36  2.03649e-48     4.51604e-66   1.40425e-20

In [ ]:
fa.x

In [ ]:
fa.ρ

In [ ]:
γ_ref[]